## Notebook 5 of 5
# Machine Learning Based Unbalance Detection of a Rotating Shaft Using Vibration Data
### Oliver Mey, Willi Neudeck, André Schneider and Olaf Enge-Rosenblatt
##### Fraunhofer IIS/EAS, Fraunhofer Institute for Integrated Circuits, Division Engineering of Adaptive Systems, Dresden, Germany

This Jupyter Notebook is part of a paper submission to the 25th IEEE International Conference on Emerging Technologies and Factory Automation, ETFA 2020. The notebook is the fifth one in a series of five freely available notebooks. It contains Python code fragments which were used to get the classification results described within the ETFA paper.

*Last Update: August 2020*

## Approach 4: Hidden Markov Model, with MFCC features

This notebook requires several dependencies, which can be installed in a conda environment using the following `environment.yml`:

~~~~~ yaml
name: hmm
channels:
  - conda-forge
  - defaults
dependencies:
  - h5py
  - hmmlearn=0.2.3
  - jupyter
  - librosa=0.7.2
  - matplotlib
  - numpy
  - pandas
  - python=3.8.*
  - scikit-learn=0.22.*
  - scipy
  - dask
  - dask-jobqueue
  - conda
  - numba=0.48.0
~~~~~
 
In addition, it is assumed that the notebook is run on a SLURM cluster (for fast hyperparameter search). It should be easy to modify the notebook to work without a cluster (see the comments in the concerned code cells).

In [ ]:
import pandas as pd
import numpy as np
import os.path
import matplotlib.pyplot as plt
import h5py
import types
import librosa.feature
import librosa.filters
import hmmlearn.hmm
import sklearn.linear_model
import sklearn.pipeline
import sklearn.preprocessing
import sklearn.metrics
import functools
import itertools
import multiprocessing
import dask_jobqueue
import dask.distributed
import zipfile

# Settings

The entire dataset (about 2.7 GB) is freely available via the Fraunhofer Fortadis data space (https://fordatis.fraunhofer.de/handle/fordatis/151.2). To reproduce the HMM/MFCC results from the paper, download the ZIP-file from Fordatis and set the `infile`-variable below to the path of the ZIP-file. The datasets includes recordings for 4 different unbalance strengths (1D/1E ... 4D/4E) as well as one recording without unbalance (0D/0E). The recordings that will be used for training can be selected using the `n_good` and `n_bad` variables.

The HMM/MFCC approach is believed to be sensitive for variations in the rotation speed. Therefore, a suitable speed range has to be selected using `rpm_lb` and `rpm_ub`. The recordings for training/development (0D ... 4D) cover the speed range approximately 630 ... 2340 rpm in steps of about 10 rpm. The recordings for evaluation (0E ... 4E) cover approximately 1060 ... 1930 rpm in steps of about 20 rpm.

To reproduce the data in the paper, the notebook has to be run multiple times, with different values for `rpm_lb` and `rpm_ub`.

In [ ]:
n_good, n_bad = '0D', '3D'  # Recordings used for training.
rpm_lb = 715  # Train model for this range of rotation speed: Lower bound, in RPM.
rpm_ub = 815  # Upper bound, in RPM.
win_len_ms = 1000  # Length of one sample, in milliseconds.

infile = 'fraunhofer_eas_dataset_for_unbalance_detection_v1.zip'  # Set this to the path of the data file!
skip = 50000  # Skip this number of measurements at the start of each recording.
f_sampl = 4096  # Sampling rate, in Hz.

# Load data

The following cells load the data from the ZIP-file. Currently, only one vibration signal is used ("Vibration_1"). The data is reshaped for convenience (one "sample" of length `win_len_ms` per row), and the samples are permuted randomly.

In [ ]:
def load_from_zfile(zfile, n):
    win_len = int(win_len_ms/1000 * f_sampl)
    with zfile.open(n + '.csv', 'r') as f:
        data = pd.read_csv(f).iloc[skip:, :]
    n = (data.shape[0]//win_len) * win_len
    data = data.iloc[:n, :]
    rpm = np.reshape(data['Measured_RPM'].values, (-1, win_len), order='C')
    vibr = np.reshape(data['Vibration_1'].values, (-1, win_len), order='C')
    ind, = np.nonzero(np.all(rpm > rpm_lb, axis=1) & np.all(rpm < rpm_ub, axis=1))
    np.random.seed(170287); ind = np.random.permutation(ind)
    return vibr[ind,:].copy()

def load_data(filename, n_good, n_bad):
    with zipfile.ZipFile(filename, 'r') as zfile:
        good = load_from_zfile(zfile, n_good)
        bad = load_from_zfile(zfile, n_bad)
    return good, bad

In [ ]:
good, bad = load_data(infile, n_good, n_bad)

In [ ]:
print('Number of "good" samples', good.shape[0])
print('Number of "bad" samples', bad.shape[0])

# Routines for Training + Testing 

In [ ]:
def train(
    f_sampl,  # Sampling rate, in Hz.
    good,  # "Good" training data (i.e. without aunbalance).
    bad,  # "Bad" training data (i.e. with unbalance).
    *,
    
    r_train = 0.5,  # Ratio of the "good" training data, that will be used
                    # to train the Hidden markov Model (the Logistic Regression
                    # will be trained using separate data).

    # The following hyperparameters will be optimized, later.
    fft_win_ms = 31.25,  # Length of one FFT window, in milliseconds.
    hop_len_ms = 8.0,  # Displacement of consecutive FFT windows, in milliseconds.
    n_mels = 15,  # Number of mel filters.
    hmm_states = 5,  # Number of states in the Hidden Markov Model.
):
    n_good = int(good.shape[0] * r_train)
    n_bad = int(bad.shape[0] * r_train)

    nfft = int(fft_win_ms/1000 * f_sampl)
    hop_len = int(hop_len_ms/1000 * f_sampl)
    mfcc_args = dict(sr=f_sampl, n_fft=nfft, n_mels=n_mels, hop_length=hop_len)

    # Step I:
    # Compute (scaled) features + train HMM for a portion of the "good" training data.
    
    tmp = [librosa.feature.mfcc(good[i, :], **mfcc_args).T
           for i in range(n_good)]
    feat_train = np.concatenate(tmp, axis=0)
    len_train = [m.shape[0] for m in tmp]

    scaler = sklearn.preprocessing.StandardScaler()
    scaler.fit(feat_train)
    model = hmmlearn.hmm.GaussianHMM(n_components=hmm_states)
    model.fit(scaler.transform(feat_train), lengths=len_train)
    
    # Step II:
    # Compute HMM output for a second portion of the "good" + "bad" training data,
    # then train a LogisticRegression on the (scaled) HMM output.

    tmp1 = [model.score(scaler.transform(
                librosa.feature.mfcc(good[i, :], **mfcc_args).T))
            for i in range(n_good, good.shape[0])]
    tmp2 = [model.score(scaler.transform(
                librosa.feature.mfcc(bad[i, :], **mfcc_args).T))
            for i in range(n_bad, bad.shape[0])]

    lr = sklearn.pipeline.make_pipeline(
        sklearn.preprocessing.StandardScaler(),
        sklearn.linear_model.LogisticRegression(),
    )
    lr.fit(
        np.concatenate([np.reshape(tmp1, (-1,1)), np.reshape(tmp2, (-1,1))], axis=0),
        np.array([0]*len(tmp1) + [1]*len(tmp2))
    )
    
    return mfcc_args, model, scaler, lr

In [ ]:
def test(models, good, bad):
    mfcc_args, model, scaler, lr = models
    
    tmp1 = [model.score(scaler.transform(
                librosa.feature.mfcc(good[i, :], **mfcc_args).T))
            for i in range(good.shape[0])]
    tmp2 = [model.score(scaler.transform(
                librosa.feature.mfcc(bad[i, :], **mfcc_args).T))
            for i in range(bad.shape[0])]

    result = lr.predict(
        np.concatenate([np.reshape(tmp1, (-1,1)), np.reshape(tmp2, (-1,1))], axis=0))
    actual = np.array([0]*len(tmp1) + [1]*len(tmp2))

    return sklearn.metrics.balanced_accuracy_score(actual, result)

The following routine performs a train/test split before performing training + test. It returns the test result.

In [ ]:
def get_score(
    f_sampl, good, bad, *,
    n_train=200,  # Number of samples for training.
    **kwargs
):
    m = train(f_sampl, good[:n_train,:], bad[:n_train,:], **kwargs)
    return test(m, good[n_train:,:], bad[n_train:,:])

# Hyperparameter-Search on cluster

We use a SLURM cluster for hyperparameter search. To run this notebook without a cluster, some cells in this section have to be modified according to the comments in the cells.

In [ ]:
# Option (a): Use SLURM cluster.
slurm = dask_jobqueue.SLURMCluster(cores=1, memory='2GB')
slurm.scale(100)
client = dask.distributed.Client(slurm)
display(client)
# Option (b): Without cluster.
#client = dask.distributed.Client()

We will compute `get_score()` for the cartesian product of the following hyperparameter settings:

In [ ]:
hmm_states_ = [1,2,3,4,5]
n_mels_ = [10,15,20] #[1,2,5,10,15]
fft_win_ms_ = [10,30,100,300]
hop_len_ms_ = [5,10,20,50]

args = list(itertools.product(hmm_states_, n_mels_, fft_win_ms_, hop_len_ms_))

In [ ]:
%%time
def f(a, good, bad):
    hmm_states, n_mels, fft_win_ms, hop_len_ms = a
    return get_score(
        f_sampl, good, bad,
        hmm_states=hmm_states,
        n_mels=n_mels,
        fft_win_ms=fft_win_ms,
        hop_len_ms=hop_len_ms,
    )

tmp1 = client.scatter(good)
tmp2 = client.scatter(bad)
scores = client.gather(client.map(f, args, good=tmp1, bad=tmp2))

In [ ]:
# Option (a): Use SLURM cluster.
slurm.scale(0)  # Stop jobs on the cluster.
# Option (b): Without cluster.
# (Just remove the `slurm.scale(0)` line above.)

We retrieve the result of the hyperparameter search:

In [ ]:
hmm_states, n_mels, fft_win_ms, hop_len_ms = args[np.argmax(scores)]
print('Best hyperparameters:', hmm_states, n_mels, fft_win_ms, hop_len_ms)

get_score(
    f_sampl, good, bad,
    hmm_states=hmm_states,
    n_mels=n_mels,
    fft_win_ms=fft_win_ms,
    hop_len_ms=hop_len_ms,
)

# Re-train best model on all training data + evaluate

In [ ]:
model = train(
    f_sampl, good, bad,
    hmm_states=hmm_states,
    n_mels=n_mels,
    fft_win_ms=fft_win_ms,
    hop_len_ms=hop_len_ms,
)

Training loss:

In [ ]:
n_good = '0D'
for n_bad in ('1D', '2D', '3D', '4D'):
    good, bad = load_data(infile, n_good, n_bad)
    score = test(model, good, bad)
    print(f'{rpm_lb} < rpm < {rpm_ub}; '
          f'"{n_good}" (n = {good.shape[0]}) vs. "{n_bad}" (n = {bad.shape[0]}):'
          f' balanced accuracy = {score:.2f}')

Testing loss:

In [ ]:
n_good = '0E'
for n_bad in ('1E', '2E', '3E', '4E'):
    good, bad = load_data(infile, n_good, n_bad)
    score = test(model, good, bad)
    print(f'{rpm_lb} < rpm < {rpm_ub}; '
          f'"{n_good}" (n = {good.shape[0]}) vs. "{n_bad}" (n = {bad.shape[0]}):'
          f' balanced accuracy = {score:.2f}')

# Properties of the model

In [ ]:
plt.imshow(model[1].transmat_)
plt.clim(0,1)
plt.xticks(np.arange(hmm_states))
plt.yticks(np.arange(hmm_states))
plt.colorbar()
plt.ylabel('Old HMM state')
plt.xlabel('New HMM state')
plt.title('State-transistion probabilities of HMM')
plt.show()

In [ ]:
plt.imshow(model[1].means_)
plt.xlabel('MFCCs (scaled + centered)')
plt.ylabel('HMM state')
plt.yticks(np.arange(hmm_states))
plt.title('Values of the MFCCs in the HMM-states')
plt.show()